<a href="https://colab.research.google.com/github/taizyayyamba-dev/data-mining-group-33/blob/main/DATA_MINING_GROUP_33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>